<a href="https://colab.research.google.com/github/zhixingc/CS542-DeepfakeDetection/blob/master/deepfake_data_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import cv2 as cv    # for capturing videos
import math   # for mathematical operations
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
import seaborn as sns
import os
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
DATA_FOLDER = '/content/drive/My Drive/Summer 2020/CS542 Deepfake Project/data/'
TRAIN_SAMPLE_FOLDER = 'dfdc_train_part_46'
#TEST_FOLDER = 'test_videos'
print(f"Train samples: {len(os.listdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)))}")


Train samples: 2203


In [5]:
train_list = list(os.listdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)))
train_list.remove('metadata.json')  #remove metadata file from training list 
len(train_list)

2202

In [6]:
def get_meta_from_json(path):
    df = pd.read_json(os.path.join(DATA_FOLDER, path, 'metadata.json'))
    df = df.T
    return df

meta_train_df = get_meta_from_json(TRAIN_SAMPLE_FOLDER)
meta_train_df.head()

,label,split,original
umkxoagtyp.mp4,FAKE,train,ruhjnjwokl.mp4
icctbkjedq.mp4,FAKE,train,gfdenskfme.mp4
muyvajjpge.mp4,FAKE,train,ijeqomctsa.mp4
zbwhxgarqw.mp4,FAKE,train,hhuakzfttr.mp4
lbgohgvmlj.mp4,FAKE,train,mdsmgzaltc.mp4


In [7]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))


In [8]:
missing_data(meta_train_df)

,label,split,original
Total,0,0,389
Percent,0,0,17.6658
Types,object,object,object


In [9]:
#check that the only missing data is in the column "original", from the videos that are real
missing_data(meta_train_df.loc[meta_train_df.label=='REAL'])

,label,split,original
Total,0,0,389
Percent,0,0,100
Types,object,object,object


In [10]:
#check that no file is missing
meta = np.array(list(meta_train_df.index))
storage = np.array([file for file in train_list if  file.endswith('mp4')])
print(f"Metadata: {meta.shape[0]}, Folder: {storage.shape[0]}")
print(f"Files in metadata and not in folder: {np.setdiff1d(meta,storage,assume_unique=False).shape[0]}")
print(f"Files in folder and not in metadata: {np.setdiff1d(storage,meta,assume_unique=False).shape[0]}")

Metadata: 2202, Folder: 2202
Files in metadata and not in folder: 0
Files in folder and not in metadata: 0


In [11]:
#current version
def images_from_video(train_list):
  for n in range(len(train_list)):
    video_path = os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER, train_list[n])
    capture_image = cv.VideoCapture(video_path) 
    count = 5
    while count>0:
      frames=random.randint(1,50)
      capture_image.set(1,frames)
      ret, frame = capture_image.read()
      if ret == False:
        continue #go to the beginning of the while loop
      label = meta_train_df.loc[video_path.split("/")[-1]].label
      train_labels.append(label)
      train_images.append(frame)
      count -= 1
  return None

In [14]:
chunks = [train_list[x:x+101] for x in range(0, len(train_list), 101)] #break the list of videos into batches

In [15]:
len(chunks)

22

In [ ]:
for c in range(len(chunks)):
  train_images = []
  train_labels = []
  images_from_video(chunks[c])
  t_i =np.asarray(train_images)
  t_l =np.asarray(train_labels)
  np.save(os.path.join(DATA_FOLDER,  "npy",'train_images_46_'+str(c)+'.npy'), t_i) #saving the file in a folder "npy" in the data folder
  np.save(os.path.join(DATA_FOLDER,  "npy",'train_labels_46_'+str(c)+'.npy'), t_l) #change the '46' to the folder you are processing
